# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import pprint

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
pprint.pprint(file_path_list)

/home/workspace
['/home/workspace/event_data/.ipynb_checkpoints/2018-11-15-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-22-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-08-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-29-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-05-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-24-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-06-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-07-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-25-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-27-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-20-events-checkpoint.csv',
 '/home/workspace/event_data/.ipynb_checkpoints/2018-11-16-events-check

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity 
                    WITH REPLICATION = 
                    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Answers

- Select statements are being created before the table creation to be able to identify primary key(partition key) and clustering keys. 
- For the data modelling, the conditions in the where clause are selected as primary keys for partitioning. 
- For the user_app_history table (question2), the clustering key being used for itemInSession column in order to have the sort of the songs.
- itemInSession column is added into the query for the evaluation of the question.
- For data extraction from the csv files, two options are being used. (I/O operation and Pandas dataframe)

In [8]:
# SELECT statements for the result check

select_query1 = "SELECT artist, song_title, song_length \
                 FROM app_history \
                 WHERE sessionId = 338 and itemInSession = 4" 

select_query2 = "SELECT artist, song_title, name, sessionId, iteminsession \
                 FROM user_app_history \
                 WHERE userid = 10 and sessionId = 182"

select_query3 =  "SELECT name \
                  FROM user_song_history \
                  WHERE song_title = 'All Hands Against His Own'"

In [9]:
# Creating Cassandra Tables 
create_query1 = "CREATE TABLE IF NOT EXISTS app_history ( artist text, \
                                                          song_title text, \
                                                          song_length double, \
                                                          sessionId int, \
                                                          itemInSession int, \
                                                          PRIMARY KEY (sessionId, itemInSession))"

create_query2 = "CREATE TABLE IF NOT EXISTS user_app_history ( userId int, \
                                                               name text, \
                                                               artist text, \
                                                               song_title text, \
                                                               sessionId int, \
                                                               itemInSession int, \
                                                               PRIMARY KEY ((userId, sessionId), itemInSession))" 

create_query3 = "CREATE TABLE IF NOT EXISTS user_song_history ( name text, \
                                                                song_title text, \
                                                                PRIMARY KEY (song_title))"

create_query_list = [create_query1, create_query2, create_query3]

for query in create_query_list:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

# Using pandas dataframe not to use i/o multiple times in three process
data = pd.read_csv(r"event_datafile_new.csv")

data['name'] = data['firstName'] + " " + data['lastName']
data = data.drop(['firstName','lastName'], axis = 1)
data.head()

,artist,gender,itemInSession,length,level,location,sessionId,song,userId,name
0,Harmonia,M,0,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26,Ryan Smith
1,The Prodigy,M,1,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26,Ryan Smith
2,Train,M,2,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26,Ryan Smith
3,Sony Wonder,M,0,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61,Samuel Gonzalez
4,Van Halen,F,2,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80,Tegan Levine


In [11]:
for row in data.itertuples():
    query = "INSERT INTO app_history (artist, song_title, song_length, sessionId, itemInSession)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (row[1], row[8], row[4], row[7], row[3]))
    
# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader) # skip header
#     for line in csvreader:
# ## TO-DO: Assign the INSERT statements into the `query` variable
#         query = "INSERT INTO app_history (artist, song_title, song_length, sessionId, itemInSession)"
#         query = query + "VALUES (%s, %s, %s, %s, %s)"
#         # TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
#         # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    rows1 = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows1:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

for row in data.itertuples():
    query = "INSERT INTO user_app_history (userId, name, artist, song_title, sessionId, itemInSession)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
    session.execute(query, (row[9], row[10] ,row[1], row[8], row[7], row[3]))

# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader)
#     for line in csvreader:
#         query = "INSERT INTO user_app_history (userId, name, artist, song_title, sessionId, itemInSession)"
#         query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
#         session.execute(query, (int(line[10]), "{} {}".format(line[1], line[4]), line[0], line[9], int(line[8]), int(line[3])))

In [14]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    rows2 = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows2:
    print(row.artist, row.song_title, row.name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

for row in data.itertuples():
    query = "INSERT INTO user_song_history (name, song_title)"
    query = query + " VALUES (%s, %s)"
    session.execute(query, (row[10], row[8]))

# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader)
#     for line in csvreader:
#         query = "INSERT INTO user_song_history (name, song_title)"
#         query = query + " VALUES (%s, %s)"
#         session.execute(query, ("{} {}".format(line[1], line[4]), line[9]))

In [16]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    rows3 = session.execute(select_query3)
except Exception as e:
    print(e)

for row in rows3:
    print(row.name)

Jacqueline Lynch


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE app_history")
session.execute("DROP TABLE user_app_history")
session.execute("DROP TABLE user_song_history ")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()